# Private GitHub Setup
1. Create a personal access token (classic) with the **repo** scope at https://github.com/settings/tokens.
2. Keep the token secret—do not share it or commit it to source control.
3. When prompted below, enter your GitHub username and paste the token; the notebook stores it only for this session.
4. If you prefer, set the variables ahead of time with `%env GITHUB_USERNAME=<user>` and `%env GITHUB_TOKEN=<token>`.
5. Similarly you can `%env PROJECT_NAME=<your github project>`

In [ ]:
%gh repo clone vmorozov/colab

In [ ]:

import getpass
import os

if "GITHUB_USERNAME" not in os.environ or not os.environ["GITHUB_USERNAME"].strip():
    os.environ["GITHUB_USERNAME"] = input("Enter your GitHub username: ").strip()

if "GITHUB_TOKEN" not in os.environ or not os.environ["GITHUB_TOKEN"].strip():
    os.environ["GITHUB_TOKEN"] = getpass.getpass("Enter your GitHub personal access token: ").strip()

if not os.environ["GITHUB_TOKEN"]:
    raise RuntimeError("GITHUB_TOKEN is required to clone the private repository.")

In [ ]:
from setup import (
    change_directory,
    clone_or_update_repo,
    ensure_project_name,
    execute_notebook,
    install_project_requirements,
    store_credentials_from_env,
)

In [ ]:
netrc_path = store_credentials_from_env()
print(f"Credentials stored in {netrc_path}")

In [ ]:
project_name = ensure_project_name()
project_dir = clone_or_update_repo("vmorozov", project_name)
print(f"Repository ready at {project_dir}")

In [ ]:
current_dir = change_directory(project_dir)
print(f"Working directory set to: {current_dir}")

index_url = install_project_requirements()
print(f"Using PyG wheel index: {index_url}")

In [ ]:
output_path = execute_notebook()
print(f"Notebook execution complete. Output saved to {output_path}")